In [41]:
!pip install openai

In [42]:
# importing the necessary module.
import pandas as pd
from openai import OpenAI

In [43]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **2024**

In [44]:
# reading the data and then storing the returned DataFrame
df = pd.read_json('/content/drive/Shareddrives/Data & Intelligence/2024/05. Data Sources/direktori_putusan/2024-case-types.json')

# printing the DataFrame
df

,propertyName1
0,Semua Direktori\n\nPerdata Agama 390852\n\nPer...


In [45]:
import re

# Extract the text from the dataframe
text_data = df['propertyName1'][0]

# Split the data into key-value pairs using regex to capture category and value
pattern = r"(?P<category>[a-zA-Z\s]+)(?P<value>\d+)"
matches = re.findall(pattern, text_data)

# Create a dictionary from the matches
parsed_data = {category.strip(): int(value) for category, value in matches}

# Create a new DataFrame from the parsed data
parsed_df = pd.DataFrame([parsed_data])

# Rename the column "Semua Direktori\n\nPerdata Agama" to "Perdata Agama"
parsed_df.rename(columns={"Semua Direktori\n\nPerdata Agama": "Perdata Agama", "TUN": "Tata Usaha Negara"}, inplace=True)

# Convert the dictionary into a dataframe with court names as rows instead of columns
parsed_df = pd.melt(parsed_df, var_name="Cases Type", value_name="Value")

# Display the parsed dataframe to the user
parsed_df

,Cases Type,Value
0,Perdata Agama,390852
1,Perdata,72635
2,Pidana Umum,47896
3,Pidana Khusus,42657
4,Tata Usaha Negara,3572
5,Perdata Khusus,2500
6,Pidana Militer,1175
7,Pajak,198


In [46]:
# reading the data and then storing the returned DataFrame
df2 = pd.read_json('/content/drive/Shareddrives/Data & Intelligence/2024/05. Data Sources/direktori_putusan/2024-month-names.json')

# printing the DataFrame
df2

,propertyName1
0,Januari 65141\n\nFebruari 66274\n\nMaret 65847...


In [47]:
# Extract the text from the dataframe
text_data_new = df2['propertyName1'][0]

# Adjust the regex pattern to account for the specific month format
pattern_new_adjusted = r"(?P<category>[a-zA-Z]+)\s+(?P<value>\d+)"

# Extract the key-value pairs from the text data using the adjusted pattern
matches_new_adjusted = re.findall(pattern_new_adjusted, text_data_new)

# Create a dictionary from the matches
parsed_data_new_adjusted = {category.strip(): int(value) for category, value in matches_new_adjusted}

# Create a new DataFrame from the parsed data
parsed_df_new_adjusted = pd.DataFrame([parsed_data_new_adjusted])

# Convert the dictionary into a dataframe with court names as rows instead of columns
parsed_df_new_adjusted = pd.melt(parsed_df_new_adjusted, var_name="Month Name", value_name="Value")

# Display the parsed dataframe to the user
parsed_df_new_adjusted

,Month Name,Value
0,Januari,65141
1,Februari,66274
2,Maret,65847
3,April,41720
4,Mei,70521
5,Juni,68922
6,Juli,77893
7,Agustus,66295
8,September,27543
9,Oktober,2


In [48]:
# reading the data and then storing the returned DataFrame
df3 = pd.read_json('/content/drive/Shareddrives/Data & Intelligence/2024/05. Data Sources/direktori_putusan/2024-court-names.json')

# printing the DataFrame
df3

,propertyName1
0,PA SURABAYA 7188\n\nPA INDRAMAYU 6262\n\nPA CI...


In [49]:
# Extract the text from the dataframe
text_data_court = df3['propertyName1'][0]

# Split the data into key-value pairs using regex to capture court name and value
pattern_court = r"(?P<category>[a-zA-Z\s]+)(?P<value>\d+)"
matches_court = re.findall(pattern_court, text_data_court)

# Create a dictionary from the matches
parsed_data_court = {category.strip(): int(value) for category, value in matches_court}

# Create a new DataFrame from the parsed data
parsed_df_court = pd.DataFrame([parsed_data_court])

# Convert the dictionary into a dataframe with court names as rows instead of columns
parsed_df_court_long = pd.DataFrame(list(parsed_data_court.items()), columns=["Court Name", "Value"])

# Display the parsed dataframe to the user
parsed_df_court_long

,Court Name,Value
0,PA SURABAYA,7188
1,PA INDRAMAYU,6262
2,PA CIBINONG,6069
3,PA TIGARAKSA,5316
4,PN SURABAYA,5316
...,...,...
919,PN Penajam,3
920,PTUN JAMBI,3
921,PN PASAMAN BARAT,2
922,PA CIBADAK,1


In [50]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key:··········


In [51]:
def find_province(court_name):
    client = OpenAI()

    # OpenAI message to get the province based on the court name
    # prompt = f"Which province is the court '{court_name}' located in?"

    # Create a chat message for the new API format
    response = client.chat.completions.create(
      model="gpt-4o",  # Use the GPT-4o
      messages=[
        {"role": "system", "content": "You are an expert in geography and legal systems of Indonesia. Your task is to identify the province where the specified court is located. Provide the name of the province in Bahasa Indonesia, and ensure that the output contains only the province name, without any additional information."},
        {"role": "user", "content": court_name}
      ],
      temperature=1,
      max_tokens=50,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0,
    )

    # Extract the response text (which should contain the province)
    province = response.choices[0].message.content.strip()
    return province

In [52]:
# prompt: show progress tqdm functio below

from tqdm import tqdm

# Apply the function to the "Court Name" column with progress bar
parsed_df_court_long['Province'] = [find_province(name) for name in tqdm(parsed_df_court_long['Court Name'], desc="Finding Provinces")]



Finding Provinces:   0%|          | 0/924 [00:00<?, ?it/s]

Finding Provinces:   0%|          | 1/924 [00:00<05:23,  2.85it/s]

Finding Provinces:   0%|          | 2/924 [00:00<05:54,  2.60it/s]

Finding Provinces:   0%|          | 3/924 [00:01<05:55,  2.59it/s]

Finding Provinces:   0%|          | 4/924 [00:01<06:23,  2.40it/s]

Finding Provinces:   1%|          | 5/924 [00:01<05:55,  2.59it/s]

Finding Provinces:   1%|          | 6/924 [00:02<06:02,  2.53it/s]

Finding Provinces:   1%|          | 7/924 [00:02<06:04,  2.52it/s]

Finding Provinces:   1%|          | 8/924 [00:03<07:37,  2.00it/s]

Finding Provinces:   1%|          | 9/924 [00:03<07:18,  2.09it/s]

Finding Provinces:   1%|          | 10/924 [00:04<06:57,  2.19it/s]

Finding Provinces:   1%|          | 11/924 [00:04<06:39,  2.29it/s]

Finding Provinces:   1%|▏         | 12/924 [00:05<06:23,  2.38it/s]

Finding Provinces:   1%|▏         | 13/924 [00:05<06:43,  2.26it/s]

Finding Provinces:   2%|▏         | 14/924 [00:06<

In [53]:
# Display the updated DataFrame
parsed_df_court_long

,Court Name,Value,Province
0,PA SURABAYA,7188,Jawa Timur
1,PA INDRAMAYU,6262,Jawa Barat
2,PA CIBINONG,6069,Jawa Barat
3,PA TIGARAKSA,5316,Banten
4,PN SURABAYA,5316,Jawa Timur
...,...,...,...
919,PN Penajam,3,Kalimantan Timur
920,PTUN JAMBI,3,Jambi
921,PN PASAMAN BARAT,2,Sumatera Barat
922,PA CIBADAK,1,Jawa Barat


In [54]:
parsed_df_court_long.to_csv('/content/parsed_df_court_long_with_province.csv')

# **2023**

In [56]:
# reading the data and then storing the returned DataFrame
df = pd.read_json('/content/drive/Shareddrives/Data & Intelligence/2024/05. Data Sources/direktori_putusan/2023-case-types.json')

# printing the DataFrame
df

,propertyName1
0,Semua Direktori\n\nPerdata Agama 609273\n\nPer...


In [57]:
import re

# Extract the text from the dataframe
text_data = df['propertyName1'][0]

# Split the data into key-value pairs using regex to capture category and value
pattern = r"(?P<category>[a-zA-Z\s]+)(?P<value>\d+)"
matches = re.findall(pattern, text_data)

# Create a dictionary from the matches
parsed_data = {category.strip(): int(value) for category, value in matches}

# Create a new DataFrame from the parsed data
parsed_df = pd.DataFrame([parsed_data])

# Rename the column "Semua Direktori\n\nPerdata Agama" to "Perdata Agama"
parsed_df.rename(columns={"Semua Direktori\n\nPerdata Agama": "Perdata Agama", "TUN": "Tata Usaha Negara"}, inplace=True)

# Convert the dictionary into a dataframe with court names as rows instead of columns
parsed_df = pd.melt(parsed_df, var_name="Cases Type", value_name="Value")

# Display the parsed dataframe to the user
parsed_df

,Cases Type,Value
0,Perdata Agama,609273
1,Perdata,112786
2,Pidana Umum,77632
3,Pidana Khusus,70488
4,Tata Usaha Negara,9448
5,Perdata Khusus,4486
6,Pidana Militer,2171
7,Pajak,762
8,Sengketa Kewenangan Mengadili,1


In [58]:
# reading the data and then storing the returned DataFrame
df2 = pd.read_json('/content/drive/Shareddrives/Data & Intelligence/2024/05. Data Sources/direktori_putusan/2023-month-names.json')

# printing the DataFrame
df2

,propertyName1
0,Januari 63655\n\nFebruari 73820\n\nMaret 71165...


In [59]:
# Extract the text from the dataframe
text_data_new = df2['propertyName1'][0]

# Adjust the regex pattern to account for the specific month format
pattern_new_adjusted = r"(?P<category>[a-zA-Z]+)\s+(?P<value>\d+)"

# Extract the key-value pairs from the text data using the adjusted pattern
matches_new_adjusted = re.findall(pattern_new_adjusted, text_data_new)

# Create a dictionary from the matches
parsed_data_new_adjusted = {category.strip(): int(value) for category, value in matches_new_adjusted}

# Create a new DataFrame from the parsed data
parsed_df_new_adjusted = pd.DataFrame([parsed_data_new_adjusted])

# Convert the dictionary into a dataframe with court names as rows instead of columns
parsed_df_new_adjusted = pd.melt(parsed_df_new_adjusted, var_name="Month Name", value_name="Value")

# Display the parsed dataframe to the user
parsed_df_new_adjusted

,Month Name,Value
0,Januari,63655
1,Februari,73820
2,Maret,71165
3,April,43779
4,Mei,76753
5,Juni,76267
6,Juli,77108
7,Agustus,84199
8,September,69673
9,Oktober,81351


In [60]:
# reading the data and then storing the returned DataFrame
df3 = pd.read_json('/content/drive/Shareddrives/Data & Intelligence/2024/05. Data Sources/direktori_putusan/2023-court-names.json')

# printing the DataFrame
df3

,propertyName1
0,MAHKAMAH AGUNG 16884\n\nPA SURABAYA 10440\n\nP...


In [61]:
# Extract the text from the dataframe
text_data_court = df3['propertyName1'][0]

# Split the data into key-value pairs using regex to capture court name and value
pattern_court = r"(?P<category>[a-zA-Z\s]+)(?P<value>\d+)"
matches_court = re.findall(pattern_court, text_data_court)

# Create a dictionary from the matches
parsed_data_court = {category.strip(): int(value) for category, value in matches_court}

# Create a new DataFrame from the parsed data
parsed_df_court = pd.DataFrame([parsed_data_court])

# Convert the dictionary into a dataframe with court names as rows instead of columns
parsed_df_court_long = pd.DataFrame(list(parsed_data_court.items()), columns=["Court Name", "Value"])

# Display the parsed dataframe to the user
parsed_df_court_long

,Court Name,Value
0,MAHKAMAH AGUNG,16884
1,PA SURABAYA,10440
2,PA CIBINONG,9910
3,PA INDRAMAYU,9535
4,PA Soreang,8649
...,...,...
924,PTA KALIMANTAN UTARA,8
925,PN Latihan,4
926,PTA JAYAPURA,4
927,PA PANIAI,2


In [62]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key:··········


In [63]:
def find_province(court_name):
    client = OpenAI()

    # OpenAI message to get the province based on the court name
    # prompt = f"Which province is the court '{court_name}' located in?"

    # Create a chat message for the new API format
    response = client.chat.completions.create(
      model="gpt-4o",  # Use the GPT-4o
      messages=[
        {"role": "system", "content": "You are an expert in geography and legal systems of Indonesia. Your task is to identify the province where the specified court is located. Provide the name of the province in Bahasa Indonesia, and ensure that the output contains only the province name, without any additional information."},
        {"role": "user", "content": court_name}
      ],
      temperature=1,
      max_tokens=50,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0,
    )

    # Extract the response text (which should contain the province)
    province = response.choices[0].message.content.strip()
    return province

In [64]:
# prompt: show progress tqdm functio below

from tqdm import tqdm

# Apply the function to the "Court Name" column with progress bar
parsed_df_court_long['Province'] = [find_province(name) for name in tqdm(parsed_df_court_long['Court Name'], desc="Finding Provinces")]



Finding Provinces:   0%|          | 0/929 [00:00<?, ?it/s]

Finding Provinces:   0%|          | 1/929 [00:00<10:16,  1.51it/s]

Finding Provinces:   0%|          | 2/929 [00:01<10:46,  1.43it/s]

Finding Provinces:   0%|          | 3/929 [00:01<09:02,  1.71it/s]

Finding Provinces:   0%|          | 4/929 [00:02<09:34,  1.61it/s]

Finding Provinces:   1%|          | 5/929 [00:03<09:45,  1.58it/s]

Finding Provinces:   1%|          | 6/929 [00:03<08:39,  1.78it/s]

Finding Provinces:   1%|          | 7/929 [00:03<07:43,  1.99it/s]

Finding Provinces:   1%|          | 8/929 [00:04<09:10,  1.67it/s]

Finding Provinces:   1%|          | 9/929 [00:05<07:59,  1.92it/s]

Finding Provinces:   1%|          | 10/929 [00:05<08:57,  1.71it/s]

Finding Provinces:   1%|          | 11/929 [00:06<08:38,  1.77it/s]

Finding Provinces:   1%|▏         | 12/929 [00:09<20:07,  1.32s/it]

Finding Provinces:   1%|▏         | 13/929 [00:09<15:28,  1.01s/it]

Finding Provinces:   2%|▏         | 14/929 [00:10<

In [65]:
# Display the updated DataFrame
parsed_df_court_long

,Court Name,Value,Province
0,MAHKAMAH AGUNG,16884,DKI Jakarta
1,PA SURABAYA,10440,Jawa Timur
2,PA CIBINONG,9910,Jawa Barat
3,PA INDRAMAYU,9535,Jawa Barat
4,PA Soreang,8649,Jawa Barat
...,...,...,...
924,PTA KALIMANTAN UTARA,8,Kalimantan Utara
925,PN Latihan,4,Daerah Khusus Ibukota Jakarta
926,PTA JAYAPURA,4,Papua
927,PA PANIAI,2,Papua Pegunungan


In [66]:
parsed_df_court_long.to_csv('/content/parsed_df_court_long_with_province.csv')

# **2022**

In [67]:
# reading the data and then storing the returned DataFrame
df = pd.read_json('/content/drive/Shareddrives/Data & Intelligence/2024/05. Data Sources/direktori_putusan/2022-case-types.json')

# printing the DataFrame
df

,propertyName1
0,Semua Direktori\n\nPerdata Agama 632332\n\nPer...


In [68]:
import re

# Extract the text from the dataframe
text_data = df['propertyName1'][0]

# Split the data into key-value pairs using regex to capture category and value
pattern = r"(?P<category>[a-zA-Z\s]+)(?P<value>\d+)"
matches = re.findall(pattern, text_data)

# Create a dictionary from the matches
parsed_data = {category.strip(): int(value) for category, value in matches}

# Create a new DataFrame from the parsed data
parsed_df = pd.DataFrame([parsed_data])

# Rename the column "Semua Direktori\n\nPerdata Agama" to "Perdata Agama"
parsed_df.rename(columns={"Semua Direktori\n\nPerdata Agama": "Perdata Agama", "TUN": "Tata Usaha Negara"}, inplace=True)

# Convert the dictionary into a dataframe with court names as rows instead of columns
parsed_df = pd.melt(parsed_df, var_name="Cases Type", value_name="Value")

# Display the parsed dataframe to the user
parsed_df

,Cases Type,Value
0,Perdata Agama,632332
1,Perdata,108866
2,Pidana Umum,80084
3,Pidana Khusus,69795
4,Tata Usaha Negara,9147
5,Perdata Khusus,5105
6,Pidana Militer,2964
7,Pajak,728


In [69]:
# reading the data and then storing the returned DataFrame
df2 = pd.read_json('/content/drive/Shareddrives/Data & Intelligence/2024/05. Data Sources/direktori_putusan/2022-month-names.json')

# printing the DataFrame
df2

,propertyName1
0,Januari 64203\n\nFebruari 67195\n\nMaret 60148...


In [70]:
# Extract the text from the dataframe
text_data_new = df2['propertyName1'][0]

# Adjust the regex pattern to account for the specific month format
pattern_new_adjusted = r"(?P<category>[a-zA-Z]+)\s+(?P<value>\d+)"

# Extract the key-value pairs from the text data using the adjusted pattern
matches_new_adjusted = re.findall(pattern_new_adjusted, text_data_new)

# Create a dictionary from the matches
parsed_data_new_adjusted = {category.strip(): int(value) for category, value in matches_new_adjusted}

# Create a new DataFrame from the parsed data
parsed_df_new_adjusted = pd.DataFrame([parsed_data_new_adjusted])

# Convert the dictionary into a dataframe with court names as rows instead of columns
parsed_df_new_adjusted = pd.melt(parsed_df_new_adjusted, var_name="Month Name", value_name="Value")

# Display the parsed dataframe to the user
parsed_df_new_adjusted

,Month Name,Value
0,Januari,64203
1,Februari,67195
2,Maret,60148
3,April,64384
4,Mei,49678
5,Juni,98487
6,Juli,78392
7,Agustus,84595
8,September,80493
9,Oktober,81012


In [71]:
# reading the data and then storing the returned DataFrame
df3 = pd.read_json('/content/drive/Shareddrives/Data & Intelligence/2024/05. Data Sources/direktori_putusan/2022-court-names.json')

# printing the DataFrame
df3

,propertyName1
0,MAHKAMAH AGUNG 22772\n\nPA SURABAYA 10184\n\nP...


In [72]:
# Extract the text from the dataframe
text_data_court = df3['propertyName1'][0]

# Split the data into key-value pairs using regex to capture court name and value
pattern_court = r"(?P<category>[a-zA-Z\s]+)(?P<value>\d+)"
matches_court = re.findall(pattern_court, text_data_court)

# Create a dictionary from the matches
parsed_data_court = {category.strip(): int(value) for category, value in matches_court}

# Create a new DataFrame from the parsed data
parsed_df_court = pd.DataFrame([parsed_data_court])

# Convert the dictionary into a dataframe with court names as rows instead of columns
parsed_df_court_long = pd.DataFrame(list(parsed_data_court.items()), columns=["Court Name", "Value"])

# Display the parsed dataframe to the user
parsed_df_court_long

,Court Name,Value
0,MAHKAMAH AGUNG,22772
1,PA SURABAYA,10184
2,PA CIBINONG,9050
3,PA JEMBER,8877
4,PN SURABAYA,8523
...,...,...
911,PA KEFAMENANU,7
912,PTA KUPANG,7
913,PA PANIAI,5
914,PN KASONGAN,5


In [73]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key:··········


In [74]:
def find_province(court_name):
    client = OpenAI()

    # OpenAI message to get the province based on the court name
    # prompt = f"Which province is the court '{court_name}' located in?"

    # Create a chat message for the new API format
    response = client.chat.completions.create(
      model="gpt-4o",  # Use the GPT-4o
      messages=[
        {"role": "system", "content": "You are an expert in geography and legal systems of Indonesia. Your task is to identify the province where the specified court is located. Provide the name of the province in Bahasa Indonesia, and ensure that the output contains only the province name, without any additional information."},
        {"role": "user", "content": court_name}
      ],
      temperature=1,
      max_tokens=50,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0,
    )

    # Extract the response text (which should contain the province)
    province = response.choices[0].message.content.strip()
    return province

In [75]:
# prompt: show progress tqdm functio below

from tqdm import tqdm

# Apply the function to the "Court Name" column with progress bar
parsed_df_court_long['Province'] = [find_province(name) for name in tqdm(parsed_df_court_long['Court Name'], desc="Finding Provinces")]


Finding Provinces: 100%|██████████| 916/916 [08:35<00:00,  1.78it/s]


In [76]:
# Display the updated DataFrame
parsed_df_court_long

,Court Name,Value,Province
0,MAHKAMAH AGUNG,22772,DKI Jakarta
1,PA SURABAYA,10184,Jawa Timur
2,PA CIBINONG,9050,Jawa Barat
3,PA JEMBER,8877,Jawa Timur
4,PN SURABAYA,8523,Jawa Timur
...,...,...,...
911,PA KEFAMENANU,7,Nusa Tenggara Timur
912,PTA KUPANG,7,Nusa Tenggara Timur
913,PA PANIAI,5,Papua
914,PN KASONGAN,5,Kalimantan Tengah


In [77]:
parsed_df_court_long.to_csv('/content/parsed_df_court_long_with_province.csv')